In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
from classes import *

In [4]:
salary = Salary(100000)

In [5]:
spendstrat = SpendingStrategy(
    base_spending=0.6,
    retirement_saving=0.15,
    disp_spend=0.15,
    disp_give=0.1
)

In [6]:
genstrat = GenerosityStrategy(
    straight_percent=0.5,
    investment_draw_down_rate=0.04,
    legacy_give_percent=0.2
)

In [7]:
pm = PortfolioManager(
    spendstrat=spendstrat,
    genstrat=genstrat,
    salary=salary,
)

In [154]:
a = Asset(
    name="a1",
    value=150000,
    growth_rate=0.038,
    dividend_rate=0.01
)

7333.3304

In [155]:
pm.taxes_ytd_ia

{1: 999.9996, 2: 999.9996, 3: 999.9996, 4: 999.9996}